## LlamaIndex and Chroma

A simple notebook largely inspired by Bhavik Jikadara in his post on medium here: https://bhavikjikadara.medium.com/llamaindex-chroma-building-a-simple-rag-pipeline-cd67fc184190

This is a one stop notebook for those that just want to test a simple RAG use case using chroma and llama3 (using Ollama)

In [ ]:
# To install these libraries, you can run the following commands:
!pip install chromadb llama-index

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-ollama
%pip install llama-index-vector-stores-chroma


In [ ]:
import sys
import subprocess
import chromadb
from llama_index.core import PromptTemplate, Settings, SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.chroma import ChromaVectorStore

## Intall Ollama

In [ ]:


def is_ollama_installed():
    """
    Check if Ollama is installed on the system.
    """
    try:
        result = subprocess.run(["ollama", "--version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            print(f"Ollama is installed: {result.stdout.decode().strip()}")
            return True
        else:
            print("Ollama is not installed.")
            return False
    except FileNotFoundError:
        print("Ollama is not installed.")
        return False

def install_ollama():
    """
    Install Ollama using the official installation script.
    """
    print("Installing Ollama...")
    try:
        # Run the installation script
        subprocess.run(
            ["curl", "-fsSL", "https://ollama.com/install.sh", "|", "sh"],
            check=True,
            shell=True,
        )
        print("Ollama installation completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during installation: {e}")
        sys.exit(1)

def verify_installation():
    """
    Verify that Ollama is installed and available.
    """
    if is_ollama_installed():
        print("Ollama is ready to use.")
    else:
        print("Failed to verify Ollama installation. Please check manually.")
        sys.exit(1)

if not is_ollama_installed():
    install_ollama()
verify_installation()

## Importing Llama LLM

With the libraries imported, we can now bring in the Llama language model. I opted for Llama because it allows for local execution, which is both free and private. Using the Ollama library makes it simple:

### Launch Ollama server

In [ ]:
import requests
import subprocess
import time

def start_ollama_server():
    try:
        subprocess.Popen(["ollama", "serve"])
        print("Ollama server started.")
    except FileNotFoundError:
        print("Error: Ollama is not installed. Please install it first.")

def pull_model(model_name):
    try:
        subprocess.run(["ollama", "pull", model_name], check=True)
        print(f"Model '{model_name}' pulled successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to pull model '{model_name}': {e}")

def is_server_running(base_url):
    try:
        response = requests.get(base_url)
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False

In [ ]:
Now we load the model 

In [ ]:

# Parameters
model_name = "llama3"
base_url = "http://localhost:11434"

# Ensure server is running
if not is_server_running(base_url):
    start_ollama_server()
    time.sleep(5)

In [ ]:
# Pull the model
pull_model(model_name)

In [ ]:
llm = Ollama(model=model_name, base_url=base_url)
response = llm.complete("Why is the sky blue?")
print(response.text)

Next, we need an embedding model to transform text into vector embeddings. I chose the “BAAI/bge-small-en-v1.5” model from Hugging Face, which is small and quick to implement — ideal for a proof of concept (POC).

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

"""
If you want something fancier or are versed in chineses, here are the available models that can be used
    BGE_MODELS = (
        "BAAI/bge-small-en",
        "BAAI/bge-small-en-v1.5",
        "BAAI/bge-base-en",
        "BAAI/bge-base-en-v1.5",
        "BAAI/bge-large-en",
        "BAAI/bge-large-en-v1.5",
        "BAAI/bge-small-zh",
        "BAAI/bge-small-zh-v1.5",
        "BAAI/bge-base-zh",
        "BAAI/bge-base-zh-v1.5",
        "BAAI/bge-large-zh",
        "BAAI/bge-large-zh-v1.5",
    )
"""


Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
import chromadb.api
import chromadb.api.client


def collection_exist(chroma_client: chromadb.api.client.Client, name: str ):
    collection_list = chroma_client.list_collections()
    collection_name_set = set((item.name for item in collection_list))
    return name in collection_name_set

In [ ]:
documents = SimpleDirectoryReader(input_files=["../data/external/Resume_Maxime_Bonnesoeur.pdf"]).load_data()
chroma_client = chromadb.EphemeralClient()
if not collection_exist(chroma_client=chroma_client, name = "ollama"):
    chroma_collection = chroma_client.create_collection("ollama")
else:
    chroma_collection = chroma_client.get_collection("ollama")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    embed_model=embed_model,
    transformations=[SentenceSplitter(chunk_size=256, chunk_overlap=10)]
)

In [ ]:
template = (
    "Imagine you are a data scientist's assistant and "
    "you answer a recruiter's questions about the data scientist's experience."
    "Here is some context from the data scientist's "
    "resume related to the query::\n"
    "-----------------------------------------\n"
    "{context_str}\n"
    "-----------------------------------------\n"
    "Considering the above information, "
    "Please respond to the following inquiry:\n\n"
    "Question: {query_str}\n\n"
    "Answer succinctly and ensure your response is "
    "clear to someone without a data science background."
    "The data scientist's name is Maxime Bonnesoeur."
)
qa_template = PromptTemplate(template)

In [ ]:
query_engine = index.as_query_engine(
    text_qa_template=qa_template,
    similarity_top_k=3
)

In [ ]:
response = query_engine.query("Do you have experience with Python?")
print(response.response)